## Mobile Game Chatbot
The purpose of this Chatbot it to recommend a game from the GooglePlayStore, based on what you like. 
It can determined this by what you say. 

For example you can say "I like to play games related to music." 
The bot then recommends a game related to music with a good rating.

It can also answer basic question as well as have some fun features.

#### Things you can ask : 
- About a game from a specific category
- Ask about time
- Ask about a fun fact
    


In [1]:
cd C:\Users\allis\Desktop\Data Science\Chatbot\data

C:\Users\allis\Desktop\Data Science\Chatbot\data


In [2]:
#pip install autocorrect

In [3]:
from tensorflow.keras.models import Sequential
from datetime import datetime
from bs4 import BeautifulSoup as bs
import requests
import random 
from tensorflow.keras.layers import Dense, Activation ,Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd 
import pickle
import numpy as np
import json
import nltk
from nltk.stem import WordNetLemmatizer
import re
from autocorrect import Speller
from termcolor import colored, cprint
nltk.download('punkt')
spell = Speller(lang ='en')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\allis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
#Creating the fact feature with web scraping
def funfact():
    r = requests.get('http://randomfactgenerator.net/')
    soup = bs(r.content)
    filtered = soup.find_all('div', attrs= {'id':'z'})
    filtered = str(filtered[0])[12:]
    filtered = filtered.split('<br/>')
    fact = filtered[0]
    return fact.lower()

In [5]:
#DataCleaning
df = pd.read_csv('googleplaystore.csv') #opening data
df.drop(df.loc[df['Category']== '1.9'].index[0] , axis = 0, inplace= True)#drop category '1.9'
df['Rating'].fillna(0,inplace=True) #drop NaNs 

df.Genres = df['Genres'].str.split(';') #split genre
df[['gen1','gen2']] = pd.DataFrame(df.Genres.tolist(), index= df.index) #split both genres

df['Genres']=  df['gen2'].fillna(df['gen1'])
df.drop(columns=['Size','Price','Current Ver','Android Ver','gen1','gen2'],inplace=True) #drop Size column
df['App'] = df['App'].str.lower() #lowercasing app column
df['Category'] = df['Category'].str.lower() #lowercasing category column
df['Genres'] = df['Genres'].str.lower() # again for the Genres column 
df['Content Rating'] = df['Content Rating'].str.lower()# same for Content rating 

df['App'] = df['App'].str.replace('[^a-zA-Z]', ' ') # removing anything that is not a letter
df['Genres'] = df['Genres'].str.replace('[^a-zA-Z]', ' ')# ^^ 
df['Content Rating'] = df['Content Rating'].str.replace('[^a-zA-Z]', ' ')#^^
df['Installs'] = df['Installs'].str.replace('[^0-9]', '')#keeping only numbers

df['Reviews'] = df['Reviews'].astype('int64') #fromating to integers
df['Installs'] = df['Installs'].astype('int64')
df['Last Updated'] = pd.to_datetime(df['Last Updated'])
df.drop_duplicates(inplace = True)

df.head(10)


<ipython-input-5-78dcd1e7d2c3>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['App'] = df['App'].str.replace('[^a-zA-Z]', ' ') # removing anything that is not a letter
<ipython-input-5-78dcd1e7d2c3>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Genres'] = df['Genres'].str.replace('[^a-zA-Z]', ' ')# ^^
<ipython-input-5-78dcd1e7d2c3>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content Rating'] = df['Content Rating'].str.replace('[^a-zA-Z]', ' ')#^^
<ipython-input-5-78dcd1e7d2c3>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Installs'] = df['Installs'].str.replace('[^0-9]', '')#keeping only numbers


,App,Category,Rating,Reviews,Installs,Type,Content Rating,Genres,Last Updated
0,photo editor candy camera grid scrapbook,art_and_design,4.1,159,10000,Free,everyone,art design,2018-01-07
1,coloring book moana,art_and_design,3.9,967,500000,Free,everyone,pretend play,2018-01-15
2,u launcher lite free live cool themes hide ...,art_and_design,4.7,87510,5000000,Free,everyone,art design,2018-08-01
3,sketch draw paint,art_and_design,4.5,215644,50000000,Free,teen,art design,2018-06-08
4,pixel draw number art coloring book,art_and_design,4.3,967,100000,Free,everyone,creativity,2018-06-20
5,paper flowers instructions,art_and_design,4.4,167,50000,Free,everyone,art design,2017-03-26
6,smoke effect photo maker smoke editor,art_and_design,3.8,178,50000,Free,everyone,art design,2018-04-26
7,infinite painter,art_and_design,4.1,36815,1000000,Free,everyone,art design,2018-06-14
8,garden coloring book,art_and_design,4.4,13791,1000000,Free,everyone,art design,2017-09-20
9,kids paint free drawing fun,art_and_design,4.7,121,10000,Free,everyone,creativity,2018-07-03


In [6]:
#making dataframe for games categories
game = df.loc[df['Category']=='game']

#merging similar Categories 
game['Genres'] = game['Genres'].str.replace('brain games', 'education')
game['Genres'] = game['Genres'].str.replace('trivia', 'education')
game['Genres'] = game['Genres'].str.replace('word', 'education')
game['Genres'] = game['Genres'].str.replace('action   adventure', 'adventure')
game['Genres'] = game['Genres'].str.replace('pretend play', 'role playing')
game['Genres'] = game['Genres'].str.replace('creativity', 'role playing')

game.to_csv(r'game.csv', index=False)
gamesort = game.groupby('Genres').sum().sort_values('Installs')
gamecategory = list(gamesort.index)
gamesort.to_csv(r'gamecategory.csv')

<ipython-input-6-9c783a432949>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game['Genres'] = game['Genres'].str.replace('brain games', 'education')
<ipython-input-6-9c783a432949>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game['Genres'] = game['Genres'].str.replace('trivia', 'education')
<ipython-input-6-9c783a432949>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [7]:
#Setting up lemmatizer, loading json file and setting up lists
lem = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())
words = []
classes = []
docs = []
punct = ['?','!','.',',']

#Lemmatizing patterns for training
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w_list = nltk.word_tokenize(pattern)
        words.extend(w_list)
        docs.append((w_list, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = sorted(set([lem.lemmatize(word) for word in words if word not in punct]))
classes =sorted(set(classes))



_train = []
_output = [0] * len(classes) #adds 0 for every classes

#bag of words
for doc in docs:
    bag_words = []
    w_patrns = doc[0]
    w_patrns = [lem.lemmatize(word.lower()) for word in w_patrns]
    for word in words:
        bag_words.append(1) if word in w_patrns else bag_words.append(0)
    
    out_row = list(_output)
    out_row[classes.index(doc[1])] = 1
    _train.append([bag_words, out_row])
    
random.shuffle(_train)

pickle.dump(words,open('words.pkl', 'wb'))
pickle.dump(classes,open('classes.pkl', 'wb'))

In [8]:
#Building model
_train = np.array(_train)
    
x_train = list(_train[:,0])
y_train = list(_train[:,1])

model = Sequential()
model.add(Dense(128,input_shape = (len(x_train[0]),), activation ='relu'))
model.add(Dropout(0.5)) # prevent overfitting 
model.add(Dense(64,activation='relu'))  
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]),activation='softmax'))#scales results to get %   

sgd = SGD(lr= 0.01, decay= 1e-6, momentum= 0.9, nesterov = True)#setting up optimizer 
model.compile(loss ='categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

brain = model.fit(np.array(x_train), np.array(y_train),epochs= 200, batch_size = 5 , verbose= 1)
model.save('brain.h5',brain)

<ipython-input-8-d8f9a0a1dacb>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  _train = np.array(_train)
C:\Users\allis\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
35/35 [==============================] - 0s 821us/step - loss: 3.0999 - accuracy: 0.0465
Epoch 2/200
35/35 [==============================] - 0s 817us/step - loss: 3.0384 - accuracy: 0.1279
Epoch 3/200
35/35 [==============================] - 0s 675us/step - loss: 2.9688 - accuracy: 0.1279
Epoch 4/200
35/35 [==============================] - 0s 704us/step - loss: 2.8927 - accuracy: 0.1802
Epoch 5/200
35/35 [==============================] - 0s 851us/step - loss: 2.8537 - accuracy: 0.1628
Epoch 6/200
35/35 [==============================] - 0s 880us/step - loss: 2.7806 - accuracy: 0.1802
Epoch 7/200
35/35 [==============================] - 0s 939us/step - loss: 2.5830 - accuracy: 0.2616
Epoch 8/200
35/35 [==============================] - 0s 792us/step - loss: 2.5101 - accuracy: 0.2384
Epoch 9/200
35/35 [==============================] - 0s 792us/step - loss: 2.3093 - accuracy: 0.2965
Epoch 10/200
35/35 [==============================] - 0s 792us/step - loss: 2.3550 - accura

35/35 [==============================] - 0s 704us/step - loss: 0.1374 - accuracy: 0.9593
Epoch 82/200
35/35 [==============================] - 0s 616us/step - loss: 0.1945 - accuracy: 0.9651
Epoch 83/200
35/35 [==============================] - 0s 675us/step - loss: 0.1396 - accuracy: 0.9651
Epoch 84/200
35/35 [==============================] - 0s 645us/step - loss: 0.1896 - accuracy: 0.9419
Epoch 85/200
35/35 [==============================] - ETA: 0s - loss: 0.0131 - accuracy: 1.00 - 0s 645us/step - loss: 0.1068 - accuracy: 0.9651
Epoch 86/200
35/35 [==============================] - 0s 616us/step - loss: 0.1274 - accuracy: 0.9593
Epoch 87/200
35/35 [==============================] - 0s 675us/step - loss: 0.1077 - accuracy: 0.9709
Epoch 88/200
35/35 [==============================] - 0s 645us/step - loss: 0.1409 - accuracy: 0.9535
Epoch 89/200
35/35 [==============================] - 0s 675us/step - loss: 0.0996 - accuracy: 0.9826
Epoch 90/200
35/35 [==============================] -

35/35 [==============================] - 0s 675us/step - loss: 0.0597 - accuracy: 0.9767
Epoch 161/200
35/35 [==============================] - 0s 645us/step - loss: 0.0539 - accuracy: 0.9826
Epoch 162/200
35/35 [==============================] - 0s 645us/step - loss: 0.0851 - accuracy: 0.9651
Epoch 163/200
35/35 [==============================] - 0s 616us/step - loss: 0.0650 - accuracy: 0.9709
Epoch 164/200
35/35 [==============================] - 0s 645us/step - loss: 0.0755 - accuracy: 0.9884
Epoch 165/200
35/35 [==============================] - 0s 616us/step - loss: 0.0587 - accuracy: 0.9884
Epoch 166/200
35/35 [==============================] - 0s 675us/step - loss: 0.0641 - accuracy: 0.9942
Epoch 167/200
35/35 [==============================] - 0s 616us/step - loss: 0.1121 - accuracy: 0.9709
Epoch 168/200
35/35 [==============================] - 0s 645us/step - loss: 0.0864 - accuracy: 0.9767
Epoch 169/200
35/35 [==============================] - 0s 645us/step - loss: 0.0684 - a

In [9]:
def inputclean(_input):
    phrase = spell(_input)
    ind_words = nltk.word_tokenize(phrase)
    words = [lem.lemmatize(word) for word in ind_words]
    return words

def bag_of_words(_input):
    _words = inputclean(_input)
    bag= [0]* len(words)
    #print(_words)
    for x in _words:
        for i, word in enumerate(words):
            if word == x:
                bag[i]= 1
    #print(bag)
    return np.array(bag)

def classpicker(_input): #this predicts what the user is saying/asking using our model
    bow = bag_of_words(_input)
    #print(bow)
    result = model.predict(np.array([bow]))[0]
    threshold = 0.25 # cap for result
    final = [[i,r] for i, r in enumerate(result) if r > threshold]
    
    final.sort(key=lambda x : x[1], reverse =True)
    return_list = []
    for r in final:
        return_list.append({"intent":classes[r[0]], "prob":str([r[1]])})
    return return_list

def response(intents_list,json_file): # outputs the correct response based on the prediction 
    neutral = ['greetings','goodbye','feel?','who?']
    tag = intents_list[0]['intent']
    _list = json_file['intents']
    for i in _list:
        if i['tag'] == tag:
            #print(tag)
            if tag in neutral:# outputs from json response bracket
                reply = random.choice(i['response'])
                break
            if tag == 'game':# outputs all games categories
                gamestr = "casino\nsimulation\ncard\nboard\nrole playing\nmusic\neducation\nstrategy\nsports\nadventure\npuzzle\nracing\ncasual\naction\narcade'"
                cprint(random.choice(i['response']),'green')
                reply = gamestr + '\n\nDoes anything interest you?'
                break
            if tag in gamecategory: # if a category is recognized it prints a random game based on rating 
                a = (random.choice(i['response']))
                b = (random.choice(gamefindr(tag)))
                ans = (a+b)
                reply = (str(ans) + '. Anything else I can help you with?')
                break
            if tag == 'time':# Out puts current time
                atime = (random.choice(i['response']))
                now = datetime.now()
                reply = atime + str(now.strftime("%H:%M"))
                break
            if tag == 'fact':# Out puts current time
                afact = (random.choice(i['response']))
                reply = afact + funfact()
                break            
            
    return reply

def gamefindr(category):# sorts games by an acceptable rating 
    tempdf = game.loc[(game.Genres == category) & (game.Rating > 4)]
    return list(tempdf.App)



    
#_input = input()
#inputclean(_input)

In [10]:
'''cprint( "ChatBot: Hi, My name is GoogleplayGames Chat bot!", 'green')
while True:
    message = input("Me: " + "")
    ints = classpicker(message)
    res = "ChatBot: " + response(ints, intents)
    cprint( res, 'green')'''

'cprint( "ChatBot: Hi, My name is GoogleplayGames Chat bot!", \'green\')\nwhile True:\n    message = input("Me: " + "")\n    ints = classpicker(message)\n    res = "ChatBot: " + response(ints, intents)\n    cprint( res, \'green\')'

In [11]:
words

["'s",
 'Can',
 'Do',
 'Games',
 'Give',
 'Hello',
 'How',
 'I',
 'Im',
 'Life',
 'RPG',
 'See',
 'Show',
 'Tell',
 'What',
 'Whats',
 'Who',
 'You',
 'a',
 'action',
 'addicting',
 'adventure',
 'adventuring',
 'ahve',
 'all',
 'alot',
 'am',
 'an',
 'and',
 'any',
 'anything',
 'app',
 'apps',
 'arcade',
 'are',
 'at',
 'athletic',
 'attack',
 'backgammon',
 'ball',
 'baseball',
 'based',
 'basketball',
 'been',
 'bit',
 'blackjack',
 'board',
 'boardgames',
 'bot',
 'brain',
 'bye',
 'can',
 'candy',
 'car',
 'card',
 'care',
 'casino',
 'casual',
 'cave',
 'chance',
 'character',
 'checker',
 'chess',
 'classic',
 'code',
 'combat',
 'created',
 'creator',
 'crush',
 'current',
 'day',
 'dice',
 'dicorvering',
 'discovering',
 'do',
 'doin',
 'dont',
 'easy',
 'eduaction',
 'education',
 'educational',
 'elearning',
 'em',
 'enemy',
 'environements',
 'explore',
 'exploring',
 'explosion',
 'fact',
 'fan',
 'fast',
 'feeling',
 'fight',
 'find',
 'for',
 'fun',
 'gamble',
 'gamblin